In [19]:
import pandas as pd
import torch
import torch.nn as nn
import random

# Exemplo simulado de um dataframe com letras
data = {'letra': ["i love you", "you're the one", "in the moonlight"]}
lyrics_df = pd.DataFrame(data)

# Pré-processamento das letras
all_lyrics = " ".join(lyrics_df['letra'])  # Concatenar todas as letras
all_lyrics = all_lyrics.lower()  # Converter para minúsculas

# Criar um dicionário de caracteres únicos presentes nas letras
chars = sorted(list(set(all_lyrics)))
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}

# Parâmetros do modelo
n_vocab = len(chars)
hidden_dim = 128
embedding_dim = 64
dropout = 0.2
seq_length = 1  # Agora estamos gerando um caractere de cada vez

# Definir a classe Simple_LSTM
class Simple_LSTM(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, dropout=0.2):
        super(Simple_LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout, num_layers=2)  # Definindo a camada LSTM
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)  # Camada de embeddings
        self.fc = nn.Linear(hidden_dim, n_vocab)  # Camada totalmente conectada para a saída
    
    def forward(self, seq_in):
        embedded = self.embeddings(seq_in)  # Obter representações de embedding diretamente
        lstm_out, _ = self.lstm(embedded.permute(1, 0, 2))  # Passar pelas camadas LSTM com permutação
        ht = lstm_out[-1]  # Pegar a saída da última etapa da LSTM
        out = self.fc(ht)  # Passar pela camada totalmente conectada para obter a saída
        return out


model = Simple_LSTM(n_vocab, hidden_dim, embedding_dim, dropout)

# Função para gerar letras
def generate_lyrics(start_text, num_verses=3, max_length=1000):
    model.eval()

    start_text = start_text.lower()
    
    input_sequence = [char_to_index[char] for char in start_text]
    input_tensor = torch.unsqueeze(torch.tensor(input_sequence), 0)
    
    generated_text = start_text
    
    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_tensor)
            probabilities = torch.nn.functional.softmax(output[0], dim=0).numpy()
            predicted_index = random.choices(range(n_vocab), weights=probabilities)[0]
            predicted_char = index_to_char[predicted_index]
            
            generated_text += predicted_char
            input_tensor = torch.unsqueeze(torch.tensor([[predicted_index]]), 0)
            
            if predicted_char == '\n':
                num_verses -= 1
                if num_verses <= 0:
                    break
    
    return generated_text

# Gerar letras para um exemplo do dataframe
start_text = "i love you"
generated_lyrics = generate_lyrics(start_text, num_verses=2)
print(generated_lyrics)


tensor([[ 5],
        [ 0],
        [ 6],
        [ 9],
        [13],
        [ 2],
        [ 0],
        [14],
        [ 9],
        [12]])
______
tensor([5])
______
tensor([[ 5],
        [ 0],
        [ 6],
        [ 9],
        [13],
        [ 2],
        [ 0],
        [14],
        [ 9],
        [12]])


RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
outinho

[]